In [1]:
import numpy as np
import paddle
import paddle.fluid as fluid
user_num = int(943)         #m
item_num = int(1682)        #n
mu = float(6e-3)
learn_rate = float(2.5)
iterations = int(20)

In [2]:
def generate(filename):
    M = np.zeros(shape=(943+1, 1682+1))
    with open(filename) as f:       # 打开指定文件
        lines = f.readlines()
        for line in lines:
            (userid, movieid, rating, ts) = line.split('\t')  # 数据集中每行有4项   用户ID，电影ID，评分，时间戳
            # 只取前三个
            uid = int(userid)
            mid = int(movieid)
            rat = float(rating)
            M[uid, mid] = rat
    # print(M)
    return M


In [3]:
def update_X(X, M):     #时间过多
    x = paddle.to_tensor(X)
    m = paddle.to_tensor(M)
    # print(x)
    # print(m)
    X = paddle.where(m > 0, m, x)
    return X

In [4]:
def update_Y(X, Z, mu):

    T = Z / mu
    T = T + X
    Y = paddle.maximum(T, paddle.zeros_like(T))
    return Y

In [5]:
def update_Z(X, Y, Z, mu):
    T = X - Y
    T = T * mu
    return Z + T

In [6]:
def update_mu(mu):
    return mu * learn_rate

In [7]:
def generate_HR_k(ans, X, M, k):
    #ans, X: m*n
    ans = paddle.to_tensor(ans)
    predicts = paddle.where(M > 0, paddle.zeros_like(M), X)
    value_1, indices_1 = paddle.topk(ans, k=k)
    value_2, indices_2 = paddle.topk(predicts, k=k)
    print(indices_1)
    print(indices_2)
    sum = 0.0
    for user in range(user_num+1):
        for j in range(k):
            # print(indices_2[user][j])
            # print(indices_1[user][:10])
            # print(indices_2[user][j] in indices_1[user][:10])
            if indices_2[user][j] in indices_1[user][:k]:
                sum += 1.0
                break
    print("HR",sum)
    return sum / user_num

In [8]:
def generate_ARHR_k(ans, X, M, k):
    #ans, X: m*n
    ans = paddle.to_tensor(ans)
    predicts = paddle.where(M > 0, paddle.zeros_like(M), X)
    value_1, indices_1 = paddle.topk(ans, k=k)
    value_2, indices_2 = paddle.topk(predicts, k=k)
    # print(indices_1)
    # print(indices_2)
    sum = 0.0
    for user in range(user_num+1):
        for j in range(k):
            # print(indices_2[user][j])
            # print(indices_1[user][:10])
            # print(indices_2[user][j] in indices_1[user][:10])
            if indices_2[user][j] in indices_1[user][:k]:
                sum += 1.0 / (j+1)
                break
    print("ARHR",sum)
    return sum / user_num

In [9]:
with fluid.dygraph.guard():
    M = generate(filename="ml-100k/u5.base")
    M = paddle.to_tensor(M)
    ans = generate(filename="ml-100k/u5.test")
    ans = paddle.to_tensor(ans)
    # print(M)
    X = paddle.ones_like(paddle.to_tensor(M))
    X = update_X(X,M)
    # print(X)
    Y = paddle.clone(M)                                              #Y0 = M
    # print(Y)
    Z = paddle.zeros(shape=(943+1, 1682+1), dtype='float64')         #Z0 = 0


In [10]:
with fluid.dygraph.guard():
    for iteration in range(iterations):
        A = Y
        A -= Z / mu
        u, s, vh = paddle.linalg.svd(A)
        # print(s)
        sigma = []
        for sigma_A in s:
            # delta = (sigma_A + 1)*(sigma_A + 1) - (4.0 / mu)
            # if delta < 0.0:
            #     sigma.append([0.0])
            # else:
            #     sigma.append(paddle.tolist((sigma_A -1 + paddle.sqrt(delta) / 2)))
            if sigma_A == 0:
                sigma.append([0.0])
            else:
                sigma.append(paddle.tolist((sigma_A - 1) / 2))

        # print(sigma)
        sigma = paddle.to_tensor(sigma).astype('float64')

        sigma = paddle.squeeze(sigma)
        # print(sigma)
        X = paddle.matmul(u, paddle.diag(sigma))
        X = paddle.matmul(X, vh)
        # X = u * paddle.diag(sigma) * vh
        X = update_X(X, M)
        Y = update_Y(X, Z, mu)
        Z = update_Z(X, Y, Z, mu)
        mu *= learn_rate
        print("finish iteration",iteration)
        # if(iteration % 100 == 0):
        #     error =
    print(generate_HR_k(ans, X, M, k=10))
    print(generate_ARHR_k(ans, X, M, k=10))

finish iteration 0
finish iteration 1
finish iteration 2
finish iteration 3
finish iteration 4
finish iteration 5
finish iteration 6
finish iteration 7
finish iteration 8
finish iteration 9
finish iteration 10
finish iteration 11
finish iteration 12
finish iteration 13
finish iteration 14
finish iteration 15
finish iteration 16
finish iteration 17
finish iteration 18
finish iteration 19
Tensor(shape=[944, 10], dtype=int64, place=Place(cpu), stop_gradient=True,
       [[8  , 9  , 4  , ..., 0  , 2  , 6  ],
        [59 , 19 , 152, ..., 111, 13 , 52 ],
        [302, 275, 286, ..., 1  , 4  , 9  ],
        ...,
        [1  , 298, 919, ..., 455, 257, 15 ],
        [197, 99 , 215, ..., 282, 97 , 71 ],
        [42 , 69 , 79 , ..., 173, 64 , 2  ]])
Tensor(shape=[944, 10], dtype=int64, place=Place(cpu), stop_gradient=True,
       [[302 , 258 , 353 , ..., 93  , 327 , 288 ],
        [231 , 172 , 474 , ..., 461 , 204 , 423 ],
        [275 , 286 , 1281, ..., 1009, 126 , 248 ],
        ...,
        [7